In [2]:
import numpy as np
import torch
import mne
import os, sys
sys.path.append(".")
sys.path.append("./code")

In [3]:
raw_data_folder = '../data/ethz_ieeg/long-term/ID01/'

In [4]:
EEG_raw_files = os.listdir(raw_data_folder)
EEG_raw_files

['ID01_1h.mat', 'ID01_2h.mat']

In [5]:
from eeg_recording import SingleSubjectRecording

In [6]:
import scipy
example_data = scipy.io.loadmat(os.path.join(raw_data_folder, EEG_raw_files[0]))['EEG']
example_data = example_data

In [24]:
mne_example_data = mne.io.RawArray(example_data, mne.create_info(ch_names=[f'eeg unknow_ch_{i}'for i in range(example_data.shape[0])], sfreq=512))

Creating RawArray with float64 data, n_channels=88, n_times=1843200
    Range : 0 ... 1843199 =      0.000 ...  3599.998 secs
Ready.


In [25]:
mne_example_data.set_channel_types(dict(zip(mne_example_data.ch_names, ['eeg'] * len(mne_example_data.ch_names))))

/tmp/ipykernel_109516/1345481350.py:1: RuntimeWarning: The unit for channel(s) eeg unknow_ch_0, eeg unknow_ch_1, eeg unknow_ch_10, eeg unknow_ch_11, eeg unknow_ch_12, eeg unknow_ch_13, eeg unknow_ch_14, eeg unknow_ch_15, eeg unknow_ch_16, eeg unknow_ch_17, eeg unknow_ch_18, eeg unknow_ch_19, eeg unknow_ch_2, eeg unknow_ch_20, eeg unknow_ch_21, eeg unknow_ch_22, eeg unknow_ch_23, eeg unknow_ch_24, eeg unknow_ch_25, eeg unknow_ch_26, eeg unknow_ch_27, eeg unknow_ch_28, eeg unknow_ch_29, eeg unknow_ch_3, eeg unknow_ch_30, eeg unknow_ch_31, eeg unknow_ch_32, eeg unknow_ch_33, eeg unknow_ch_34, eeg unknow_ch_35, eeg unknow_ch_36, eeg unknow_ch_37, eeg unknow_ch_38, eeg unknow_ch_39, eeg unknow_ch_4, eeg unknow_ch_40, eeg unknow_ch_41, eeg unknow_ch_42, eeg unknow_ch_43, eeg unknow_ch_44, eeg unknow_ch_45, eeg unknow_ch_46, eeg unknow_ch_47, eeg unknow_ch_48, eeg unknow_ch_49, eeg unknow_ch_5, eeg unknow_ch_50, eeg unknow_ch_51, eeg unknow_ch_52, eeg unknow_ch_53, eeg unknow_ch_54, eeg unkno

Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,Not available
Good channels,88 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,512.00 Hz
Highpass,0.00 Hz
Lowpass,256.00 Hz


In [16]:
mne_example_data = mne_example_data.set_eeg_reference(ref_channels="average")

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


In [19]:
data = mne.filter.filter_data(mne_example_data.get_data(), sfreq = 512, l_freq = 0.5, h_freq=70)

Setting up band-pass filter from 0.5 - 70 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 70.00 Hz
- Upper transition bandwidth: 17.50 Hz (-6 dB cutoff frequency: 78.75 Hz)
- Filter length: 3381 samples (6.604 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


In [20]:
mne_example_data = mne.io.RawArray(data, mne_example_data.info)

Creating RawArray with float64 data, n_channels=47, n_times=248320
    Range : 0 ... 248319 =      0.000 ...   484.998 secs
Ready.


In [7]:
recording = SingleSubjectRecording(0 << 8 + 2, mne_example_data)

NameError: name 'mne_example_data' is not defined

In [8]:
from sklearn.decomposition import PCA
from hmmlearn import hmm
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt
def embedx(x, lags):
    """Embeds a [samples * channels] array X using a vector of time lags
    resulting in a [channels * samples * lags] array xe.
    
    Parameters
    ----------
    x : array
        Input data [channels x samples].
    lags : list
        The desired lags.

    Return
    ------
    xe : array
        The embedded data [channels, samples, lags]. xe contains
        only the valid subsection of the orignal data, after edge
        effects have been removed.
    valid : array
        Boolean array indexing the valid samples in xe.
    """
    Xe = np.zeros((x.shape[1], x.shape[0],  len(lags)))

    for l in range(len(lags)):
        Xe[:, :, l] = np.roll(x, lags[l], axis=0).swapaxes(0, 1)

    # Remove edges
    valid = np.ones((x.shape[0], 1), dtype=np.int8)
    valid[:np.abs(np.min(lags)), :] = 0
    valid[-np.abs(np.max(lags)):, :] = 0

    Xe = Xe[:, valid[:, 0] == 1, :]

    return Xe, valid


def hmm_tde(data: np.array, lags, n_states=3, n_components=8, **kwargs):
    """Time-delay embedded Hidden markov model.

    Adapted from the HMM-MAR toolbox [3].

    Parameters
    ----------
    data : array
        Observations [channels * samples].
    lags : array | list
        Sample lags.
    n_states : int
        Number of states.
    n_components : int
        Number of components for the PCQ decomposition.

    Return
    ------
    gamma : array
        Time courses of the states probabilities given data.
    model : hmmlearn object.
        Hidden Markov Model.
    xe : array
        Embedded data [channels * samples * lags]

    References
    ----------
    [1] Quinn, A. J., van Ede, F., Brookes, M. J., Heideman, S. G., Nowak, M., Seedat,
        Z. A., … Woolrich, M. W. (2019). Unpacking Transient Event Dynamics in
        Electrophysiological Power Spectra. Brain Topography, 32(6), 1020–1034.
        https://doi.org/10.1007/s10548-019-00745-5

    [2] Vidaurre, D., Hunt, L. T., Quinn, A. J., Hunt, B. A. E., Brookes, M. J.,
        Nobre, A. C., & Woolrich, M. W. (2018). Spontaneous cortical activity
        transiently organises into frequency specific phase-coupling networks.
        Nature Communications, 9(1). https://doi.org/10.1038/s41467-018-05316-z

    [3] https://github.com/OHBA-analysis/HMM-MAR
    """

    # Embed time serie
    xe, valid = embedx(data, lags)

    pca = PCA(n_components=n_components)
    y = pca.fit_transform(xe[0, :, :])

    model = hmm.GaussianHMM(n_components=n_states, n_iter=100,
                            covariance_type='full', **kwargs)
    model.fit(y)
    gamma = model.predict_proba(y)
    
    return gamma, model, xe

In [9]:
data = mne_example_data.get_data()

NameError: name 'mne_example_data' is not defined

In [ ]:
xe, valid = embedx(data, lags)


In [36]:
gamma, model, xe = hmm_tde(, [[15]] * 88, 6)

: 

In [27]:
recording.run_latent_hmm(n_states=9, use_gfp=True)

Model is not converging.  Current: -46790717.86348778 is not greater than -46790717.83350316. Delta is -0.029984623193740845


In [28]:
microstates = recording.latent_maps
microstates.shape

(9, 88)

In [29]:
recording.gev_tot

np.float64(0.0064375399111945225)

In [221]:
data = mne_example_data.get_data()

In [180]:
from data_utils import corr_vectors, get_gfp_peaks

activation = microstates.dot(data)
segmentation = np.argmax(np.abs(activation), axis=0)
print(activation)

gfp_curve = None
(gfp_peeks, gfp_curve) = get_gfp_peaks(
    data, min_peak_dist=2, smoothing=None, smoothing_window=100
)
gfp_sum_sq = np.sum(gfp_curve**2)
map_corr = corr_vectors(data, microstates[segmentation].T)

gev_tot = sum((gfp_curve * map_corr) ** 2) / gfp_sum_sq
gev_tot



[[ 30.78629073  32.38387228  32.53269755 ... 184.13942999 159.92214329
  137.17126715]
 [-32.93767442 -38.67300829 -47.76458465 ... -25.41480863 -34.60075786
  -42.55006125]
 [ -5.43664283   4.14588945  17.33972476 ... 362.70422868 329.65057379
  298.06995285]
 ...
 [-70.90081658 -73.39596684 -80.83949145 ... -99.93996995 -96.75787581
  -95.07881456]
 [-71.20564574 -69.23512586 -74.24628047 ... -64.02229791 -58.65491401
  -53.85131512]
 [-36.10473468 -39.34547843 -43.1381015  ...  16.56768132  25.29825924
   35.16601051]]


0.489340963862868

In [276]:
from data_utils import get_gfp_peaks
class MicrostateTrainingModel(torch.nn.Module):
    
    def __init__(self, n_microstate, n_channels, W = None):
        super(MicrostateTrainingModel, self).__init__()
        weight_shape = (n_microstate, n_channels)
        if W is None:
            W = np.random.random(weight_shape)
            W = torch.DoubleTensor(W)
            norm = W.pow(2).sum(dim=1).sqrt()
            W = W / norm.unsqueeze(-1)
            self.W = torch.nn.Linear(n_microstate, n_channels, bias  = False)
            self.W.weight = torch.nn.Parameter(torch.DoubleTensor(W))
        else:
            self.W = torch.nn.Linear(n_microstate, n_channels, bias  = False)
            self.W.weight = torch.nn.Parameter(torch.DoubleTensor(W))
            
        self.loss_function = torch.nn.MSELoss()
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.0001, betas=(0.9, 0.999))

            
    
    def corr_vectors(self, A, B, axis=0):
        An = A - torch.mean(A, dim=axis, keepdims=True)
        Bn = B - torch.mean(B, dim=axis, keepdims=True)
        new_An = An / torch.linalg.norm(An, dim=axis, keepdims=True)
        new_Bn = Bn / torch.linalg.norm(Bn, dim=axis, keepdims=True)
        return torch.sum(new_An * new_Bn, dim=axis)
    
    def forward(self, eeg_data):
        
        microstates_maps = self.W
        eeg_data = torch.DoubleTensor(eeg_data)
        activation = torch.abs(microstates_maps(eeg_data.T)).T
        segmentation = torch.argmax(activation, axis = 0)
        return segmentation
        
    def loss(self, eeg_data, segmentation, **kwargs):
        # loss
        (peaks, gfp_curve) = get_gfp_peaks(
            eeg_data,
            min_peak_dist=kwargs.pop("min_peak_dist", 2),
            smoothing=kwargs.pop("smoothing", None),
            smoothing_window=kwargs.pop("smoothing_window", 100),
        )
        gfp_sum_sq = np.sum(gfp_curve ** 2)

        eeg_data = torch.DoubleTensor(eeg_data)
        map_corr = self.corr_vectors(eeg_data, self.W.weight[segmentation].T)
        
        
        gev = torch.sum((torch.DoubleTensor(gfp_curve) * map_corr) ** 2) / gfp_sum_sq
        
        loss = self. loss_function(gev, torch.DoubleTensor([1]))
        print(loss, gev)
        return loss
        

In [277]:
model = MicrostateTrainingModel(n_microstate=31, n_channels=47, W = microstates)

In [278]:
data = mne_example_data.get_data()

In [279]:
segmentation = model.forward(data)

In [280]:
running_loss = 0
for i  in range(10000):
        # Zero your gradients for every batch!
        model.optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model.forward(data)

        # Compute the loss and its gradients
        loss = model.loss(data, outputs)
        loss.backward()

        # Adjust learning weights
        model.optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        print(loss)
        
        with torch.no_grad():
                for param in model.parameters():
                        norm = param.pow(2).sum(dim=1).sqrt()
                        param = param / norm.unsqueeze(-1)

tensor(0.3497, dtype=torch.float64, grad_fn=<MseLossBackward0>) tensor(0.4086, dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(0.3497, dtype=torch.float64, grad_fn=<MseLossBackward0>)
tensor(0.3497, dtype=torch.float64, grad_fn=<MseLossBackward0>) tensor(0.4087, dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(0.3497, dtype=torch.float64, grad_fn=<MseLossBackward0>)
tensor(0.3496, dtype=torch.float64, grad_fn=<MseLossBackward0>) tensor(0.4087, dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(0.3496, dtype=torch.float64, grad_fn=<MseLossBackward0>)
tensor(0.3496, dtype=torch.float64, grad_fn=<MseLossBackward0>) tensor(0.4087, dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(0.3496, dtype=torch.float64, grad_fn=<MseLossBackward0>)
tensor(0.3497, dtype=torch.float64, grad_fn=<MseLossBackward0>) tensor(0.4087, dtype=torch.float64, grad_fn=<DivBackward0>)
tensor(0.3497, dtype=torch.float64, grad_fn=<MseLossBackward0>)
tensor(0.3497, dtype=torch.float64, grad_fn=<MseLossBackward

KeyboardInterrupt: 